In [ ]:
from scipy import special
import scipy
import math
import numpy as np
import meep as mp
import meep.adjoint as mpa
import autograd.numpy as npa
from autograd import tensor_jacobian_product, grad
from matplotlib import pyplot as plt
from matplotlib.patches import Circle
import matplotlib.pyplot as plt
import cmath
import meep as mp
from IPython.display import Video
import nlopt
import os

# direction= ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25',]
direction= ['1', '2', '3', '4', '5', '6', '7', '8', '9']

AR2= [0, 0.030, 0.060, 0.090, 0.120, 0.150, 0.180, 0.210, 0.240]

Design_region_H= [0.210 * 5 + 0.120, 0.240 * 5 + 0.120, 0.270 * 5 + 0.120, 0.300 * 5 + 0.120, 0.330 * 5 + 0.120, 0.340 * 5 + 0.120, 0.390 * 5 + 0.120]

Lens_Rad= [2.520, 2.520]

Focal_L= [0.510, 166.7]

AR1= [0, 0.030, 0.060, 0.090, 0.120, 0.150, 0.180, 0.210, 0.240]


Focal_size= [0.630*4, 35]

Res= [50, 25]

Incidence_angle= [0, 10, 34]

is_get_dft_free= False

###### Code 접기용 ########
Load_design= False      ##
free_space= True        ##
material= True          ##
Source_activate= True   ##
##########################

# scaling & refractive index
um_scale = 1/0.750 # 1A = 675nm

# Si= mp.Medium(index=1.4) # SiO2
# Air= mp.Medium(index=1) # SiPD
# HfO2= mp.Medium(index=1.9) # HfO2
# SiPD= mp.Medium(index=11.8) # SiPD

Air = mp.Medium(index=1.0)
SiN = mp.Medium(index=2.1)
SiO2 = mp.Medium(index=1.4)
Design = mp.Medium(index=1.75)
HfO2 = mp.Medium(index=1.9)
SiPD = mp.Medium(index=4.7)

for dir in range(9, 9+1):

    design_dir="./"+direction[dir-1]+"/"
    if not os.path.exists(design_dir):
        try:
            os.makedirs(design_dir)
        except:
            print('e')

    os.chdir(direction[dir-1])
    """ Meep unit: 10um """

    resolution = Res[0]
    Lpml = 0.40 # PML 영역 크기
    pml_2_source = 0.20
    source_2_geo = 0.20

    # 높이 (z: thickness)
    ARt_h= round(AR1[dir-1] * um_scale,2) # 2 layer 1/4+ 2/4
    Design_region_h= round(Design_region_H[0] * um_scale,2) # multi layer size 210 ~ 390 nm
    Focal_length= round(Focal_L[0] * um_scale,2) # focal layer size 510 ~ nm
    # ARb_h= round(AR2[0] * um_scale,2) # 3 layer 1/4+ 2/4+ 1/4
    ARb_h = 0
    SiPD_h = 0


    # 폭 (r: width)
    Lens_R= round(Lens_Rad[0] * um_scale,2) # 반경: 40 lambda
    # Sensor_R= 0.854* Lens_R

    # 전체 공간 비례 변수 (pml 뺀 유효 공간)
    Sx= Lens_R # 반경: 4 mm X 4 mm
    Sy= round(pml_2_source+ source_2_geo+ ARt_h+ Design_region_h + Focal_length + ARb_h + SiPD_h,2) # 위에서부터 아래로 쌓기
    

    Y_min= 0.5*-Sy
    Y_max= 0.5*Sy 
    X_min= 0.5*-Sx
    X_max= 0.5*Sx 

    # 구조 정보
    geometry= [
        mp.Block(#AR coating top side (Core)
            center= mp.Vector3(0, round(Y_max- pml_2_source- source_2_geo- 0.5*ARt_h,2)),
            material= SiO2,
            size= mp.Vector3(Sx, ARt_h)
        ),
        mp.Block(#designregion
            center= mp.Vector3(0, round(Y_max- pml_2_source- source_2_geo- ARt_h- 0.5*Design_region_h,2)),
            material= Design,
            size= mp.Vector3(Sx, Design_region_h)
        ),
        mp.Block(#focal_length
            center= mp.Vector3(0, round(Y_max- pml_2_source- source_2_geo- ARt_h- Design_region_h - 0.5*Focal_length,2)),
            material= SiO2,
            size= mp.Vector3(Sx, Focal_length)
        ),
        mp.Block(#AR coating bottom side (Core)
            center= mp.Vector3(0, round(Y_max- pml_2_source- source_2_geo- ARt_h- Design_region_h - Focal_length - 0.5*ARb_h,2)),
            material= HfO2,
            size= mp.Vector3(Sx, ARb_h)
        ),
        mp.Block(#SiPD(Core)
            center= mp.Vector3(0, round(Y_max- pml_2_source- source_2_geo- ARt_h- Design_region_h - Focal_length - ARb_h- SiPD_h,2)),
            material= SiO2,
            size= mp.Vector3(Sx, round(2*SiPD_h,2))
        ),
    ]

    # 전체공간
    cell= mp.Vector3(Sx, Sy+ 2*Lpml) # <-- pml 두께는 여기서 추가
    pml_layers = [mp.PML(thickness=Lpml, direction=mp.Y)]

    # 대역
    fcen= 1/(0.455*um_scale)
    fwidth= 0.4 *fcen
    simtime= 30
    nfrq = 1
    #src=mp.ContinuousSource(frequency=fcen, fwidth=fwidth, is_integrated=True)
    src = mp.GaussianSource(frequency=fcen, fwidth=fwidth, is_integrated=True)
    for theta_i in range(0,3):
        theta= Incidence_angle[theta_i] # 입사각, degree로 정의
        if Source_activate:
            # Oblique (20 + -20 deg)
            TM= []
            k_z= 2* np.pi* fcen* 1.4* np.sin(np.radians(theta)) # k vector of cartatian
            xxx= X_max- 1/resolution # initial position of point source\
            while xxx >= X_min+(1/resolution): # multiple point sources
                amplitude= npa.exp(1j*k_z*xxx)
                TM.append(
                            mp.Source(
                                src,
                                component=mp.Hz,
                                center=mp.Vector3(xxx, round(Y_max- pml_2_source,2)),
                                amplitude = amplitude,
                            ),
                        )
                xxx-=1/resolution

            # Oblique (20 + -20 deg)
            TE= []
            k_x= 2* np.pi* fcen* 1.4* np.sin(np.radians(theta)) # k vector of cartatian
            xxx= X_max- 1/resolution # initial position of point source\
            while xxx >= X_min+(1/resolution): # multiple point sources
                amplitude= npa.exp(1j*k_x*xxx)
                TE.append(
                            mp.Source(
                                src,
                                component=mp.Ez,
                                center=mp.Vector3(xxx, round(Y_max- pml_2_source,2)),
                                amplitude = amplitude,
                            ),
                        )
                xxx-=1/resolution

        #kpoint= mp.Vector3(1,0)
        kpoint = mp.Vector3(0,-fcen*1.4).rotate(mp.Vector3(z=1), np.radians(theta))
        Source_region= round(pml_2_source+ source_2_geo,2)
        Flux_center= mp.Vector3(0, round(Y_max- Source_region+ 2/resolution,2))
        L_center= mp.Vector3(0, round(Y_max- pml_2_source- source_2_geo- ARt_h- Design_region_h - Focal_length - ARb_h + 2/resolution,2))   
        tot_size= mp.Vector3(Sx, 0, 0)

        ### TE mode
        if free_space :
            simTE = mp.Simulation(
                cell_size=cell,
                resolution=resolution,
                boundary_layers=pml_layers,
                sources=TE,
                k_point=kpoint,
                #geometry=geometry,
                default_material=Air,
                # symmetries=[mp.Mirror(mp.X)] if rot_angle == 0 else [],
            )
            # input flux
            incidence_flux = simTE.add_flux(fcen, fwidth, 1, mp.FluxRegion(center=Flux_center, size=tot_size))

            pt = L_center

            # simTE.run(until_after_sources=mp.stop_when_fields_decayed(10,mp.Ez,pt,1e-3))

            f = plt.figure(dpi=150)
            volume = mp.Block(size=mp.Vector3(Sx,Sy,0), center=mp.Vector3(0))
            Animate = mp.Animate2D(simTE, output_plane=volume, fields=mp.Ez, f=f, realtime=False, normalize=True)
            simTE.run(mp.at_every(0.1, Animate), until=simtime)

            filename = "test.mp4"
            Animate.to_mp4(10,filename)
            Video(filename)

            # simTE.run(until_after_sources=simtime)
            # simTE.run(until=200)
            refl_flux = simTE.get_flux_data(incidence_flux)
            Input_flux = mp.get_fluxes(incidence_flux)
            plt.figure(dpi=200)
            simTE.plot2D(fields=mp.Ez)
            plt.savefig("Free_TE_Ez"+str(theta)+".png")
            plt.close()

        if material :
            simTE.reset_meep()
            simTE = mp.Simulation(
                cell_size=cell,
                resolution=resolution,
                boundary_layers=pml_layers,
                sources=TE,
                k_point=kpoint,
                geometry=geometry,
                default_material=Air,
                # symmetries=[mp.Mirror(mp.X)] if rot_angle == 0 else [],
            )

            # reflected flux
            refl = simTE.add_flux(fcen, fwidth, 1, mp.FluxRegion(center=Flux_center, size=tot_size))
            simTE.load_minus_flux_data(refl, refl_flux) # free-space 값을 빼서 back scattering 제거
            # transmitted flux on output port of lens
            L_tran = simTE.add_flux(fcen, fwidth, 1, mp.FluxRegion(center=L_center, size=tot_size)) # 렌즈 Transmission

            plt.figure(dpi=150)
            simTE.plot2D(output_plane = mp.Volume(center=mp.Vector3(0,0,0), size = mp.Vector3(Sx, Sy, 0)))
            plt.savefig("HfO2_TE"+str(theta)+".png")
            plt.close()

            # simTE.run(until_after_sources=mp.stop_when_fields_decayed(10,mp.Ez,pt,1e-3))

            simTE.run(until_after_sources=simtime)

            reflection = mp.get_fluxes(refl)
            Lens_flux = mp.get_fluxes(L_tran)

            plt.figure(dpi=200)
            simTE.plot2D(fields=mp.Ez)
            plt.savefig("HfO2_TE_Ez"+str(theta)+".png")
            plt.close()
            Rs = []
            Ts = []

            Rs = np.append(Rs, -reflection[0] / Input_flux[0])
            Ts = np.append(Ts, Lens_flux[0] / Input_flux[0])

            print('reflection =',Rs[0]*100,'%')
            print('transmission =',Ts[0]*100),'%'
            print('Loss =',(1-Rs[0]-Ts[0])*100,'%')

            loss_val=[Rs[0]*100, Ts[0]*100, (1-Rs[0]-Ts[0])*100]

            np.savetxt("AR_TR_TE"+str(theta) +".txt", loss_val)
            simTE.reset_meep()

        if free_space :
            ### TM mode
            simTM = mp.Simulation(
                cell_size=cell,
                resolution=resolution,
                boundary_layers=pml_layers,
                sources=TM,
                k_point=kpoint,
                #geometry=geometry,
                default_material=Air,
                # symmetries=[mp.Mirror(mp.X)] if rot_angle == 0 else [],
            )
            # input flux
            incidence_flux = simTM.add_flux(fcen, fwidth, 1, mp.FluxRegion(center=Flux_center, size=tot_size))
            # simTM.run(until_after_sources=mp.stop_when_fields_decayed(10,mp.Hz,pt,1e-3))
            simTM.run(until_after_sources=simtime/3.5)
            # simTM.run(until=200)
            refl_flux = simTM.get_flux_data(incidence_flux)
            Input_flux = mp.get_fluxes(incidence_flux)
            plt.figure(dpi=200)
            simTM.plot2D(fields=mp.Hz)
            plt.savefig("Free_TM_Hz"+str(theta)+".png")
            plt.close()

        if material : 
            simTM.reset_meep()
            simTM = mp.Simulation(
                cell_size=cell,
                resolution=resolution,
                boundary_layers=pml_layers,
                sources=TM,
                k_point=kpoint,
                geometry=geometry,
                default_material=Air,
                # symmetries=[mp.Mirror(mp.X)] if rot_angle == 0 else [],
            )

            # reflected flux
            refl = simTM.add_flux(fcen, fwidth, 1, mp.FluxRegion(center=Flux_center, size=tot_size))
            simTM.load_minus_flux_data(refl, refl_flux) # free-space 값을 빼서 back scattering 제거
            # transmitted flux on output port of lens
            L_tran = simTM.add_flux(fcen, fwidth, 1, mp.FluxRegion(center=L_center, size=tot_size)) # 렌즈 Transmission

            plt.figure(dpi=150)
            simTM.plot2D(output_plane = mp.Volume(center=mp.Vector3(0,0,0), size = mp.Vector3(Sx, Sy, 0)))
            plt.savefig("HfO2_TM"+str(theta)+".png")

            # simTM.run(until_after_sources=mp.stop_when_fields_decayed(10,mp.Hz,pt,1e-3))

            simTM.run(until_after_sources=simtime)

            reflection = mp.get_fluxes(refl)
            Lens_flux = mp.get_fluxes(L_tran)

            plt.figure(dpi=200)
            simTM.plot2D(fields=mp.Hz)
            plt.savefig("HfO2_TM_Hz"+str(theta)+".png")
            plt.close()
            Rs = []
            Ts = []

            Rs = np.append(Rs, -reflection[0] / Input_flux[0])
            Ts = np.append(Ts, Lens_flux[0] / Input_flux[0])

            print('reflection =',Rs[0]*100,'%')
            print('transmission =',Ts[0]*100),'%'
            print('Loss =',(1-Rs[0]-Ts[0])*100,'%')

            loss_val=[Rs[0]*100, Ts[0]*100, (1-Rs[0]-Ts[0])*100]

            np.savetxt("AR_TR_TM"+str(theta) +".txt", loss_val)
            simTM.reset_meep()

    os.chdir("..")
